In [2]:
import scipy.spatial.distance
import pandas as pd
import numpy as np
import re
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim import models
from gensim.corpora import Dictionary
from timeit import default_timer as timer

dictionary = Dictionary()
tfidf_gensim = models.TfidfModel()
model_words = None

def tokenizeText(string):
    #prepares the string for tokenization, to lowercase, then removes punctutation, then changes numbers to _NUM_
    string = string.lower()
    string = re.sub(r"\d+\.?\d*", " _NUM_ ", string)
    string = re.sub(r'[^\w\s]', '',string)
    stop_words = set(stopwords.words("english"))
    tokenized = word_tokenize(string)
    return [word for word in tokenized if not word.lower() in stop_words]

C:\Users\mocka\mambaforge\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
fiveThsnd= r"C:\Users\mocka\PycharmProjects\Patent-Prior-Art-Finder\Patent Queries\5000PatentsAbstractAndCitationsJson.json"
texts = pd.io.json.read_json(fiveThsnd)["Abstract"]
tokens=  texts.apply(tokenizeText)



In [4]:
# makeModel
dictionary.add_documents(tokens)
model_words = Word2Vec(tokens,vector_size=50)
model_words = model_words
model_words.build_vocab(tokens, update=True)
model_words.train(tokens, total_examples=model_words.corpus_count,
                       epochs=model_words.epochs)

(1400397, 1658985)

In [5]:
#def _makeEmbeddings(self, texts):
corpus = [dictionary.doc2bow(line) for line in tokens]
tfidf_gensim = models.TfidfModel(corpus)
tfidfs = [tfidf_gensim[corpus[x]] for x in range(0, len(corpus))]
print(tfidfs)
for (tokenList, tfidfList) in zip(tokens, tfidfs):
    sum_words = np.empty(50)
    tfidfDict = dict(tfidfList)
    passed = []
    for word in tokenList:
        index = dictionary.token2id.get(word)
        tfidfValue = tfidfDict.get(index)
        try:
            # print("Old sum: "+str(sum_words))
            # print("Add vec:"+str(model_words.wv[word]))
            sum_words= np.add(sum_words ,model_words.wv[word])
            # sum_tfidf= np.add(sum_tfidf,np.multiply(tfidfValue,model_words[word]))
        except: # In case the model does not have a given word, ignore it.
            # passed.append(word)
            pass

In [6]:
print(sum_words)

[  26823.95824092   87917.07700482  -31000.18015315   -3837.62837662
  -66693.49937803 -167775.54740636  163713.01905292  169371.18769634
 -239843.15263559 -126935.59131568  -40969.99876656  -83896.51712935
  153097.09607382   -9548.09797845   22372.57646579  130595.95710454
  119992.849142      8554.89321494 -112015.46578098 -195798.18403545
   22455.25981647  110577.530933    183451.68448647   24752.24633699
   87433.45837898  -57138.65418347  -71492.57394365   28184.22833259
  -63177.31899117    9517.86552159  -89365.10699338   46658.22958924
   -1772.65675238  -86130.60062513  -41313.36659045  130814.46232528
  219936.54811237   62720.5547332    79150.55704734  -71548.02752986
  194168.93131773  -10045.05468765   28976.1038082   -21952.03224302
  274404.75067883  -50379.90663195  -12035.41209057  -29473.38974775
   47924.64787452  110093.40681841]


In [8]:
print(tfidfs)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

